**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [2]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
from IPython.display import Video
import skvideo.io
import cv2
import json
import itertools

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense, Flatten
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [3]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [4]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function act returns an action given a state s. The agent will explore with a probability $\epsilon$ and exploit with a propbability 1-$\epsilon$.

The exploration part is essential because the agent learns about the actions and potential rewards by exploring. Without this part, the agent would only take actions it knows with a view to maximizing the expected reward, which is exploitation.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [26]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [27]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=2 # set small when debugging
epochs_test=2 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The array board models the whole island at state s: it contains every cell with it's current reward (-1 if it is poisonous, +0.5 if it has cheese, 0 if the rat has previously passed by the cell or if it is on the cell).
The array position defines the "view" of the rat

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [38]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        action = np.random.randint(0, self.n_action)
        return action

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [28]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
    
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [53]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')

HTML(display_videos('random1.mp4'))


Win/lose count 12.5/15.0. Average score (-2.5)
Win/lose count 10.5/15.0. Average score (-3.5)
Final score: -3.5


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




\begin{equation*}
Q^{\pi}(s,a)=E_{\pi}[\overset{\infty}{\underset{k=0}{\Sigma}} \gamma^{k}r_{t+k+1} | s_{t}=s, a_{t}=a] \\ 
\Leftrightarrow Q^{\pi}(s,a)=r(s, a) + E_{\pi}[\overset{\infty}{\underset{k=1}{\Sigma}} \gamma^{k}r_{t+k+1} | s_{t}=s, a_{t}=a] \\
\end{equation*}

by linearity of the expectation

\begin{equation*}
\Leftrightarrow Q^{\pi}(s,a)=r(s, a) + \gamma \underset{s', a'}{\Sigma}P(s_{t+1}=s',a_{t+1}=a'|s_{t}=s, a_{t}=a)E_{\pi}[\overset{\infty}{\underset{k=1}{\Sigma}} \gamma^{k-1}r_{t+k} | s_{t+1}=s', a_{t+1}=a']
\end{equation*}

by conditional expectation

\begin{equation*}
\Leftrightarrow Q^{\pi}(s,a)=r(s, a) + \gamma \underset{s', a'}{\Sigma}P(s_{t+1}=s',a_{t+1}=a'|s_{t}=s, a_{t}=a)E_{\pi}[\overset{\infty}{\underset{k=0}{\Sigma}} \gamma^{k}r_{t+k+1} | s_{t+1}=s', a_{t+1}=a']
\end{equation*}

By changing the indices.
However, 

\begin{equation*}
E_{\pi}[\overset{\infty}{\underset{k=0}{\Sigma}} \gamma^{k}r_{t+k+1} | s_{t+1}=s', a_{t+1}=a'] = Q^{\pi}(s',a')
\end{equation*}

So, by linearity of the expectation we have

\begin{equation*}
\Leftrightarrow Q^{\pi}(s,a)=E_{(s', a') \sim p(.|s, a))}[r(s, a) + \gamma Q^{\pi}(s',a')]
\end{equation*}


Under assumption that $\pi^*$ exists, we have:

$a^{*} = \underset{a'}{argmax}Q^{\pi}(s', a')$

And, $E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]$ is maximized for $Q^{\pi}(s',a') = Q^{*}(s',a')$

So, by taking $\pi = \pi^{*}$ we have $Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma \max_{a'} Q^{*}(s',a')]$

The L2 norm is convex on $\mathbb{R}^{2}$ so, by using Jensen's inequality we can write: 

$E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2} \geq \Vert E_{s'\sim \pi^*(.|s,a)}[ r +\gamma \max \max_{a'}Q(s',a',\theta)-Q(s,a,\theta)]\Vert^{2} \geq \Vert E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma \max_{a'} Q^{*}(s',a')] \Vert^{2}$

So by using $\mathcal{L}(\theta)$ as our objective function, in case of convergence, we force our Q value to verify the Bellman equation

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [29]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        if len(self.memory) >= self.max_memory:
            self.memory = self.memory[-self.max_memory:]

    def random_access(self):
        return self.memory[np.random.randint(0, len(self.memory))]

***
The pipeline we will use for training is given below:

In [30]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [31]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s.reshape([1,s.shape[0],s.shape[1],s.shape[2]]))[0,:])

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            
            s_, n_s_, a_, r_, game_over_  = self.memory.random_access()
            
            target_q[i] =self.model.predict(s_.reshape([1,s_.shape[0],s_.shape[1],s_.shape[2]]))[0]
            
            if game_over_:
                
                input_states[i] = s_
                target_q[i,a_] = r_
                
            else:
                input_states[i] = s_
                target_q[i,a_] = r_  + self.discount*max(self.model.predict(n_s_.reshape([1,
                                            s_.shape[0],s_.shape[1],s_.shape[2]]))[0])
                
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        model = Sequential()
        
        model.add(Flatten(input_shape=(5,5,self.n_state,)))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In order to find the best combination of architecture and parameters, we tried the following architectures:

- DQN_FC: 2 dense layers (sizes 64 and 4), with relu activation on the intermediate layer
- DQN_FC_2layers: 3 dense layers (sizes 64, 64 and 4), with relu activation on the two intermediate layers
- DSN_FC_3layers: 4 dense layers (sizes 64, 128, 64 and 4), with relu activation on the three intermediate layers

For each of these models, we tried to optimize the learning rate, the batch_size and the memory_size parameters trying every combination of the following set of values:

- learning_rate_ : [0.1, 0.3, 0.5]
- batch_size : [32, 64]
- memory_size : [1000, 2000, 4000]

We evaluated our models with the mean test score over 50 test epochs.

You can find below the five best combinations for each architecture:

\begin{array}{|l|l|l|l|r|}
\hline \text{architecture} & \text{learning rate} & \text{batch size} & \text{memory size} & \text{mean score} \\\hline
  \text{DQN_FC} & 0.1 & 32 & 1000 & 9.08 \\\hline
  \text{DQN_FC} & 0.3 & 32 & 2000 & 7.63 \\\hline
  \text{DQN_FC} & 0.1 & 32 & 2000 & 7.28 \\\hline
  \text{DQN_FC} & 0.1 & 64 & 1000 & 6.93 \\\hline
  \text{DQN_FC} & 0.5 & 32 & 4000 & 6.87 \\\hline
  \text{DQN_FC_2layers} & 0.1 & 32 & 1000 & 7.62 \\\hline
  \text{DQN_FC_2layers} & 0.3 & 64 & 4000 & 7.37 \\\hline
  \text{DQN_FC_2layers} & 0.3 & 64 & 1000 & 7.08 \\\hline
  \text{DQN_FC_2layers} & 0.1 & 64 & 2000 & 6.97 \\\hline
  \text{DQN_FC_2layers} & 0.1 & 64 & 1000 & 6.75 \\\hline
  \text{DQN_FC_3layers} & 0.3 & 64 & 1000 & 7.88 \\\hline
  \text{DQN_FC_3layers} & 0.1 & 64 & 2000 & 7.71 \\\hline
  \text{DQN_FC_3layers} & 0.5 & 32 & 2000 & 6.96 \\\hline
  \text{DQN_FC_3layers} & 0.1 & 32 & 2000 & 6.68 \\\hline
  \text{DQN_FC_3layers} & 0.5 & 32 & 4000 & 6.56 \\\hline
\end{array}


To conclude, we finally chose a model composed of 2 dense layers (sizes 64 and 4), with relu activation on the intermediate layer, a learning rate of 0.1, a batch size of 32 and a memory size of 1000, which had a mean test error of 9.08

In [100]:
epochs_train=51 # set small when debugging
epochs_test=50 # set small when debugging

env = Environment(grid_size=size, max_time=T, temperature=0.3)

learning_rate_list = [0.1, 0.3, 0.5]
batch_size_list = [32, 64]
memory_size = [1000, 2000, 4000]

for lr, batch_size, memory_size in itertools.product(*[learning_rate_list, batch_size_list, memory_size]):
    print('Learning rate: {}, Batch size: {}, Memory Size: {}'.format(lr, batch_size, memory_size))
    
    agent = DQN_FC(size, lr=lr, epsilon = 0.1, memory_size=memory_size, batch_size = batch_size)
    train(agent, env, epochs_train, prefix='fc_train')
    #HTML(display_videos('fc_train50.mp4'))
    
    test(agent, env, epochs_test, prefix='fc_test')

Learning rate: 0.1, Batch size: 32, Memory Size: 1000
Epoch 000/051 | Loss 0.0051 | Win/lose count 6.0/5.0 (1.0)
Epoch 001/051 | Loss 0.0069 | Win/lose count 3.5/7.0 (-3.5)
Epoch 002/051 | Loss 0.0129 | Win/lose count 3.0/1.0 (2.0)
Epoch 003/051 | Loss 0.0166 | Win/lose count 4.0/6.0 (-2.0)
Epoch 004/051 | Loss 0.0052 | Win/lose count 3.0/1.0 (2.0)
Epoch 005/051 | Loss 0.0033 | Win/lose count 5.0/1.0 (4.0)
Epoch 006/051 | Loss 0.0105 | Win/lose count 4.5/4.0 (0.5)
Epoch 007/051 | Loss 0.0035 | Win/lose count 3.5/2.0 (1.5)
Epoch 008/051 | Loss 0.0013 | Win/lose count 1.0/3.0 (-2.0)
Epoch 009/051 | Loss 0.0051 | Win/lose count 3.0/2.0 (1.0)
Epoch 010/051 | Loss 0.0041 | Win/lose count 5.0/0 (5.0)
Epoch 011/051 | Loss 0.0040 | Win/lose count 3.0/2.0 (1.0)
Epoch 012/051 | Loss 0.0019 | Win/lose count 6.0/1.0 (5.0)
Epoch 013/051 | Loss 0.0073 | Win/lose count 4.5/2.0 (2.5)
Epoch 014/051 | Loss 0.0036 | Win/lose count 5.5/3.0 (2.5)
Epoch 015/051 | Loss 0.0054 | Win/lose count 5.0/2.0 (3.0)
E

Epoch 040/051 | Loss 0.0039 | Win/lose count 5.5/5.0 (0.5)
Epoch 041/051 | Loss 0.0032 | Win/lose count 7.5/3.0 (4.5)
Epoch 042/051 | Loss 0.0034 | Win/lose count 4.5/0 (4.5)
Epoch 043/051 | Loss 0.0041 | Win/lose count 8.0/1.0 (7.0)
Epoch 044/051 | Loss 0.0062 | Win/lose count 10.0/5.0 (5.0)
Epoch 045/051 | Loss 0.0020 | Win/lose count 9.0/3.0 (6.0)
Epoch 046/051 | Loss 0.0022 | Win/lose count 9.0/3.0 (6.0)
Epoch 047/051 | Loss 0.0040 | Win/lose count 5.0/5.0 (0.0)
Epoch 048/051 | Loss 0.0023 | Win/lose count 16.5/2.0 (14.5)
Epoch 049/051 | Loss 0.0038 | Win/lose count 11.0/2.0 (9.0)
Epoch 050/051 | Loss 0.0507 | Win/lose count 13.5/3.0 (10.5)
Win/lose count 12.0/3.0. Average score (9.0)
Win/lose count 5.0/1.0. Average score (6.5)
Win/lose count 17.0/4.0. Average score (8.666666666666666)
Win/lose count 18.5/2.0. Average score (10.625)
Win/lose count 6.5/3.0. Average score (9.2)
Win/lose count 9.0/5.0. Average score (8.333333333333334)
Win/lose count 10.5/3.0. Average score (8.2142857

Win/lose count 7.0/0. Average score (6.147058823529412)
Win/lose count 13.0/1.0. Average score (6.314285714285714)
Win/lose count 7.5/2.0. Average score (6.291666666666667)
Win/lose count 6.0/0. Average score (6.283783783783784)
Win/lose count 6.0/1.0. Average score (6.25)
Win/lose count 4.5/0. Average score (6.205128205128205)
Win/lose count 14.0/5.0. Average score (6.275)
Win/lose count 3.0/0. Average score (6.195121951219512)
Win/lose count 7.5/5.0. Average score (6.107142857142857)
Win/lose count 2.0/0. Average score (6.011627906976744)
Win/lose count 8.5/3.0. Average score (6.0)
Win/lose count 5.0/2.0. Average score (5.933333333333334)
Win/lose count 13.5/1.0. Average score (6.076086956521739)
Win/lose count 10.0/1.0. Average score (6.138297872340425)
Win/lose count 15.5/2.0. Average score (6.291666666666667)
Win/lose count 9.0/3.0. Average score (6.285714285714286)
Win/lose count 6.5/2.0. Average score (6.25)
Final score: 6.25
Learning rate: 0.1, Batch size: 64, Memory Size: 1000

Epoch 024/051 | Loss 0.0038 | Win/lose count 5.0/1.0 (4.0)
Epoch 025/051 | Loss 0.0249 | Win/lose count 8.0/4.0 (4.0)
Epoch 026/051 | Loss 0.0022 | Win/lose count 4.5/0 (4.5)
Epoch 027/051 | Loss 0.0488 | Win/lose count 3.5/1.0 (2.5)
Epoch 028/051 | Loss 0.0311 | Win/lose count 8.5/1.0 (7.5)
Epoch 029/051 | Loss 0.0306 | Win/lose count 0.5/0 (0.5)
Epoch 030/051 | Loss 0.0245 | Win/lose count 8.0/3.0 (5.0)
Epoch 031/051 | Loss 0.0031 | Win/lose count 3.0/1.0 (2.0)
Epoch 032/051 | Loss 0.0284 | Win/lose count 5.5/4.0 (1.5)
Epoch 033/051 | Loss 0.0302 | Win/lose count 5.5/2.0 (3.5)
Epoch 034/051 | Loss 0.0561 | Win/lose count 5.5/1.0 (4.5)
Epoch 035/051 | Loss 0.0049 | Win/lose count 8.0/6.0 (2.0)
Epoch 036/051 | Loss 0.0025 | Win/lose count 11.0/1.0 (10.0)
Epoch 037/051 | Loss 0.0333 | Win/lose count 5.5/2.0 (3.5)
Epoch 038/051 | Loss 0.0063 | Win/lose count 7.5/7.0 (0.5)
Epoch 039/051 | Loss 0.0058 | Win/lose count 5.0/1.0 (4.0)
Epoch 040/051 | Loss 0.0249 | Win/lose count 5.5/2.0 (3.5)

Win/lose count 6.0/2.0. Average score (2.5277777777777777)
Win/lose count 6.5/5.0. Average score (2.473684210526316)
Win/lose count 4.0/0. Average score (2.55)
Win/lose count 12.5/2.0. Average score (2.9285714285714284)
Win/lose count 4.0/0. Average score (2.977272727272727)
Win/lose count 2.0/3.0. Average score (2.8043478260869565)
Win/lose count 3.5/2.0. Average score (2.75)
Win/lose count 7.0/4.0. Average score (2.76)
Win/lose count 8.0/4.0. Average score (2.8076923076923075)
Win/lose count 2.0/5.0. Average score (2.5925925925925926)
Win/lose count 1.5/0. Average score (2.5535714285714284)
Win/lose count 2.5/0. Average score (2.5517241379310347)
Win/lose count 8.0/1.0. Average score (2.7)
Win/lose count 2.0/0. Average score (2.6774193548387095)
Win/lose count 3.0/3.0. Average score (2.59375)
Win/lose count 2.0/1.0. Average score (2.5454545454545454)
Win/lose count 9.0/6.0. Average score (2.5588235294117645)
Win/lose count 9.5/2.0. Average score (2.7)
Win/lose count 4.0/2.0. Average 

...

Epoch 033/051 | Loss 0.0548 | Win/lose count 6.0/3.0 (3.0)
Epoch 034/051 | Loss 0.0229 | Win/lose count 7.0/4.0 (3.0)
Epoch 035/051 | Loss 0.0032 | Win/lose count 11.5/4.0 (7.5)
Epoch 036/051 | Loss 0.0041 | Win/lose count 9.0/1.0 (8.0)
Epoch 037/051 | Loss 0.0247 | Win/lose count 4.0/0 (4.0)
Epoch 038/051 | Loss 0.0036 | Win/lose count 5.0/1.0 (4.0)
Epoch 039/051 | Loss 0.0031 | Win/lose count 8.5/5.0 (3.5)
Epoch 040/051 | Loss 0.0025 | Win/lose count 16.5/2.0 (14.5)
Epoch 041/051 | Loss 0.0025 | Win/lose count 15.0/4.0 (11.0)
Epoch 042/051 | Loss 0.0043 | Win/lose count 10.5/2.0 (8.5)
Epoch 043/051 | Loss 0.0041 | Win/lose count 17.0/3.0 (14.0)
Epoch 044/051 | Loss 0.0041 | Win/lose count 21.5/3.0 (18.5)
Epoch 045/051 | Loss 0.0251 | Win/lose count 12.0/1.0 (11.0)
Epoch 046/051 | Loss 0.0283 | Win/lose count 12.0/6.0 (6.0)
Epoch 047/051 | Loss 0.0038 | Win/lose count 5.5/3.0 (2.5)
Epoch 048/051 | Loss 0.0258 | Win/lose count 5.0/5.0 (0.0)
Epoch 049/051 | Loss 0.0039 | Win/lose count 

Win/lose count 5.0/3.0. Average score (6.173076923076923)
Win/lose count 6.0/2.0. Average score (6.092592592592593)
Win/lose count 1.0/0. Average score (5.910714285714286)
Win/lose count 7.5/0. Average score (5.9655172413793105)
Win/lose count 8.0/2.0. Average score (5.966666666666667)
Win/lose count 8.0/1.0. Average score (6.0)
Win/lose count 8.0/5.0. Average score (5.90625)
Win/lose count 10.0/4.0. Average score (5.909090909090909)
Win/lose count 7.0/2.0. Average score (5.882352941176471)
Win/lose count 6.0/3.0. Average score (5.8)
Win/lose count 10.0/0. Average score (5.916666666666667)
Win/lose count 7.5/4.0. Average score (5.851351351351352)
Win/lose count 10.0/5.0. Average score (5.828947368421052)
Win/lose count 2.0/1.0. Average score (5.705128205128205)
Win/lose count 2.5/4.0. Average score (5.525)
Win/lose count 7.0/2.0. Average score (5.512195121951219)
Win/lose count 5.5/2.0. Average score (5.464285714285714)
Win/lose count 10.5/2.0. Average score (5.534883720930233)
Win/los

Epoch 017/051 | Loss 0.0049 | Win/lose count 4.5/3.0 (1.5)
Epoch 018/051 | Loss 0.0041 | Win/lose count 7.5/0 (7.5)
Epoch 019/051 | Loss 0.0019 | Win/lose count 9.5/3.0 (6.5)
Epoch 020/051 | Loss 0.0026 | Win/lose count 11.5/4.0 (7.5)
Epoch 021/051 | Loss 0.0050 | Win/lose count 8.0/1.0 (7.0)
Epoch 022/051 | Loss 0.0023 | Win/lose count 7.0/3.0 (4.0)
Epoch 023/051 | Loss 0.0608 | Win/lose count 7.5/4.0 (3.5)
Epoch 024/051 | Loss 0.0040 | Win/lose count 4.0/2.0 (2.0)
Epoch 025/051 | Loss 0.0584 | Win/lose count 10.5/0 (10.5)
Epoch 026/051 | Loss 0.0027 | Win/lose count 6.0/4.0 (2.0)
Epoch 027/051 | Loss 0.0016 | Win/lose count 18.5/3.0 (15.5)
Epoch 028/051 | Loss 0.0619 | Win/lose count 10.0/4.0 (6.0)
Epoch 029/051 | Loss 0.0054 | Win/lose count 11.0/2.0 (9.0)
Epoch 030/051 | Loss 0.0019 | Win/lose count 11.5/3.0 (8.5)
Epoch 031/051 | Loss 0.0027 | Win/lose count 9.0/4.0 (5.0)
Epoch 032/051 | Loss 0.0063 | Win/lose count 6.0/2.0 (4.0)
Epoch 033/051 | Loss 0.0042 | Win/lose count 8.0/1.0

Win/lose count 7.0/2.0. Average score (6.166666666666667)
Win/lose count 8.5/3.0. Average score (6.1)
Win/lose count 13.5/1.0. Average score (6.681818181818182)
Win/lose count 10.5/2.0. Average score (6.833333333333333)
Win/lose count 14.5/5.0. Average score (7.038461538461538)
Win/lose count 12.5/2.0. Average score (7.285714285714286)
Win/lose count 7.5/2.0. Average score (7.166666666666667)
Win/lose count 10.5/1.0. Average score (7.3125)
Win/lose count 5.5/3.0. Average score (7.029411764705882)
Win/lose count 8.0/3.0. Average score (6.916666666666667)
Win/lose count 7.5/5.0. Average score (6.684210526315789)
Win/lose count 3.5/2.0. Average score (6.425)
Win/lose count 16.5/4.0. Average score (6.714285714285714)
Win/lose count 4.0/0. Average score (6.590909090909091)
Win/lose count 14.5/4.0. Average score (6.760869565217392)
Win/lose count 13.5/4.0. Average score (6.875)
Win/lose count 10.5/2.0. Average score (6.94)
Win/lose count 2.0/2.0. Average score (6.673076923076923)
Win/lose co

Epoch 000/051 | Loss 0.0143 | Win/lose count 5.0/5.0 (0.0)
Epoch 001/051 | Loss 0.0071 | Win/lose count 3.0/3.0 (0.0)
Epoch 002/051 | Loss 0.0028 | Win/lose count 2.5/2.0 (0.5)
Epoch 003/051 | Loss 0.0009 | Win/lose count 1.0/1.0 (0.0)
Epoch 004/051 | Loss 0.0052 | Win/lose count 4.0/2.0 (2.0)
Epoch 005/051 | Loss 0.0026 | Win/lose count 1.5/3.0 (-1.5)
Epoch 006/051 | Loss 0.0284 | Win/lose count 2.0/6.0 (-4.0)
Epoch 007/051 | Loss 0.0070 | Win/lose count 6.5/8.0 (-1.5)
Epoch 008/051 | Loss 0.0026 | Win/lose count 4.0/1.0 (3.0)
Epoch 009/051 | Loss 0.0266 | Win/lose count 1.5/2.0 (-0.5)
Epoch 010/051 | Loss 0.0022 | Win/lose count 2.0/5.0 (-3.0)
Epoch 011/051 | Loss 0.0264 | Win/lose count 8.5/1.0 (7.5)
Epoch 012/051 | Loss 0.0267 | Win/lose count 3.0/3.0 (0.0)
Epoch 013/051 | Loss 0.0024 | Win/lose count 1.5/3.0 (-1.5)
Epoch 014/051 | Loss 0.0154 | Win/lose count 7.5/4.0 (3.5)
Epoch 015/051 | Loss 0.0091 | Win/lose count 3.5/7.0 (-3.5)
Epoch 016/051 | Loss 0.0090 | Win/lose count 3.5/

In [140]:
epochs_train=51 # set small when debugging
epochs_test=50 # set small when debugging

env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=0.1, epsilon = 0.1, memory_size=1000, batch_size = 32)
train(agent,env,epochs_train,prefix='fc_train')
HTML(display_videos('fc_train50.mp4'))

Epoch 000/051 | Loss 0.0151 | Win/lose count 11.0/5.0 (6.0)
Epoch 001/051 | Loss 0.0266 | Win/lose count 5.5/4.0 (1.5)
Epoch 002/051 | Loss 0.0120 | Win/lose count 1.5/2.0 (-0.5)
Epoch 003/051 | Loss 0.0065 | Win/lose count 3.0/6.0 (-3.0)
Epoch 004/051 | Loss 0.0091 | Win/lose count 3.5/1.0 (2.5)
Epoch 005/051 | Loss 0.0155 | Win/lose count 6.0/6.0 (0.0)
Epoch 006/051 | Loss 0.0030 | Win/lose count 7.0/2.0 (5.0)
Epoch 007/051 | Loss 0.0101 | Win/lose count 1.5/3.0 (-1.5)
Epoch 008/051 | Loss 0.0048 | Win/lose count 4.0/4.0 (0.0)
Epoch 009/051 | Loss 0.0010 | Win/lose count 3.5/2.0 (1.5)
Epoch 010/051 | Loss 0.0078 | Win/lose count 7.5/8.0 (-0.5)
Epoch 011/051 | Loss 0.0075 | Win/lose count 4.0/5.0 (-1.0)
Epoch 012/051 | Loss 0.0600 | Win/lose count 3.5/0 (3.5)
Epoch 013/051 | Loss 0.0006 | Win/lose count 4.0/1.0 (3.0)
Epoch 014/051 | Loss 0.0085 | Win/lose count 1.0/1.0 (0.0)
Epoch 015/051 | Loss 0.0075 | Win/lose count 5.5/4.0 (1.5)
Epoch 016/051 | Loss 0.0035 | Win/lose count 10.5/3.

In [142]:
test(agent, env, epochs_test, prefix='fc_test')
HTML(display_videos('fc_test49.mp4'))

Win/lose count 10.5/2.0. Average score (8.5)
Win/lose count 9.0/5.0. Average score (6.25)
Win/lose count 3.5/2.0. Average score (4.666666666666667)
Win/lose count 3.0/1.0. Average score (4.0)
Win/lose count 4.5/1.0. Average score (3.9)
Win/lose count 15.0/1.0. Average score (5.583333333333333)
Win/lose count 9.0/3.0. Average score (5.642857142857143)
Win/lose count 7.5/2.0. Average score (5.625)
Win/lose count 8.5/1.0. Average score (5.833333333333333)
Win/lose count 6.0/6.0. Average score (5.25)
Win/lose count 8.0/3.0. Average score (5.2272727272727275)
Win/lose count 3.5/2.0. Average score (4.916666666666667)
Win/lose count 1.5/0. Average score (4.653846153846154)
Win/lose count 5.0/3.0. Average score (4.464285714285714)
Win/lose count 9.0/3.0. Average score (4.566666666666666)
Win/lose count 5.5/2.0. Average score (4.5)
Win/lose count 7.0/1.0. Average score (4.588235294117647)
Win/lose count 10.5/3.0. Average score (4.75)
Win/lose count 8.0/1.0. Average score (4.868421052631579)
Win

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [33]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        
        model.add(Conv2D(32, (2,2), input_shape=(5,5,self.n_state,), activation='relu'))
        model.add(Conv2D(64, (2,2), activation='relu'))
        model.add(Flatten())
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

Following a similar process than the one we used for the DQN_FC, we compared four achitectures of convolutional neural networks:

- DQN_CNN: 2 convolution layers (depths 32 and 64, 3 by 3 filters and relu activation) followed by a dense layer (size 4)
- DQN_CNN_Connected: 2 convolution layers (depths 32 and 64, 3 by 3 filters and relu activation) followed by two dense layers (sizes 64 and 4) with relu activation on the first dense layer
- DQN_CNN_22: 2 convolution layers (depths 32 and 64, 2 by 2 filters and relu activation) followed by a dense layer (size 4)
- DQN_CNN_22_Connected: 2 convolution layers (depths 32 and 64, 2 by 2 filters and relu activation) followed by two dense layers (sizes 64 and 4) with relu activation on the first dense layer

We evaluated our models with the mean test score over 50 test epochs.

You can find below the test result for each model:

\begin{array}{|l|l|l|l|r|}
\hline \text{architecture} & \text{mean score} \\\hline
  \text{DQN_CNN} & 8.45 \\\hline
  \text{DQN_CNN_Connected} & 8.26 \\\hline
  \text{DQN_CNN_22} & 9.89 \\\hline
  \text{DQN_CNN_22_Connected} & 7.48 \\\hline
\end{array}


To conclude, we finally chose a model composed of 2 convolution layers (depths 32 and 64, 2 by 2 filters and relu activation) followed by a dense layer (size 4)

In [147]:
epochs_train=51 # set small when debugging
epochs_test=50 # set small when debugging

env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train50.mp4'))

Epoch 000/051 | Loss 0.0007 | Win/lose count 4.5/0 (4.5)
Epoch 001/051 | Loss 0.0034 | Win/lose count 2.0/6.0 (-4.0)
Epoch 002/051 | Loss 0.0416 | Win/lose count 2.5/2.0 (0.5)
Epoch 003/051 | Loss 0.0120 | Win/lose count 4.0/3.0 (1.0)
Epoch 004/051 | Loss 0.0272 | Win/lose count 3.0/2.0 (1.0)
Epoch 005/051 | Loss 0.0091 | Win/lose count 4.5/4.0 (0.5)
Epoch 006/051 | Loss 0.0024 | Win/lose count 3.0/8.0 (-5.0)
Epoch 007/051 | Loss 0.0056 | Win/lose count 5.0/3.0 (2.0)
Epoch 008/051 | Loss 0.0461 | Win/lose count 3.5/2.0 (1.5)
Epoch 009/051 | Loss 0.0015 | Win/lose count 7.0/1.0 (6.0)
Epoch 010/051 | Loss 0.0435 | Win/lose count 14.0/2.0 (12.0)
Epoch 011/051 | Loss 0.0044 | Win/lose count 4.0/5.0 (-1.0)
Epoch 012/051 | Loss 0.0030 | Win/lose count 4.5/1.0 (3.5)
Epoch 013/051 | Loss 0.0027 | Win/lose count 10.5/1.0 (9.5)
Epoch 014/051 | Loss 0.0051 | Win/lose count 4.0/1.0 (3.0)
Epoch 015/051 | Loss 0.0053 | Win/lose count 14.0/2.0 (12.0)
Epoch 016/051 | Loss 0.0037 | Win/lose count 8.0/2

In [148]:
test(agent, env, epochs_test, prefix='cnn_test')
HTML(display_videos('cnn_test49.mp4'))

Win/lose count 12.0/4.0. Average score (8.0)
Win/lose count 20.0/4.0. Average score (12.0)
Win/lose count 9.5/2.0. Average score (10.5)
Win/lose count 10.5/2.0. Average score (10.0)
Win/lose count 7.5/1.0. Average score (9.3)
Win/lose count 6.5/1.0. Average score (8.666666666666666)
Win/lose count 14.0/2.0. Average score (9.142857142857142)
Win/lose count 12.0/3.0. Average score (9.125)
Win/lose count 16.0/1.0. Average score (9.777777777777779)
Win/lose count 8.0/2.0. Average score (9.4)
Win/lose count 16.5/2.0. Average score (9.863636363636363)
Win/lose count 13.0/1.0. Average score (10.041666666666666)
Win/lose count 11.0/5.0. Average score (9.73076923076923)
Win/lose count 7.0/1.0. Average score (9.464285714285714)
Win/lose count 7.5/0. Average score (9.333333333333334)
Win/lose count 11.5/4.0. Average score (9.21875)
Win/lose count 14.5/4.0. Average score (9.294117647058824)
Win/lose count 4.5/1.0. Average score (8.972222222222221)
Win/lose count 17.5/1.0. Average score (9.36842105

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [42]:
env = Environment(grid_size=size, max_time=T,temperature=0.9)

agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=0.1, epsilon = 0.1, memory_size=1000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 68.0/3.0. Average score (65.0)
Win/lose count 59.0/2.0. Average score (61.0)
Win/lose count 44.5/1.0. Average score (55.166666666666664)
Win/lose count 38.0/4.0. Average score (49.875)
Win/lose count 65.5/1.0. Average score (52.8)
Win/lose count 61.5/3.0. Average score (53.75)
Win/lose count 44.5/5.0. Average score (51.714285714285715)
Win/lose count 55.0/2.0. Average score (51.875)
Win/lose count 55.5/0. Average score (52.27777777777778)
Win/lose count 53.5/0. Average score (52.4)
Win/lose count 56.0/2.0. Average score (52.54545454545455)
Win/lose count 56.0/2.0. Average score (52.666666666666664)
Win/lose count 68.0/1.0. Average score (53.76923076923077)
Win/lose count 61.0/2.0. Average score (54.142857142857146)
Win/lose count 71.0/2.0. Average score (55.13333333333333)
Win/lose count 70.0/2.0. Average score (55.9375)
Win/lose count 62.0/3.0. Average score (56.11764705882353)
Win/lose count 48.5/0. Average score (55.69444444444444)
Win/lose count 37.5/

In [43]:
HTML(display_videos('cnn_test49.mp4'))

In [44]:
HTML(display_videos('fc_test49.mp4'))

We observed the impact of temperature on both of our best models (the CNN and the FC), the results are listed below:

\begin{array}{|l|l|l|}
\hline \text{temperature} & \text{DQN_CNN performance} & \text{DQN_FC performance} \\\hline
  0.1 & 3.18 & 2.77 \\\hline
  0.2 & 7.33 & 4.69 \\\hline
  0.3 & 9.88 & 7.58 \\\hline
  0.4 & 13.65 & 9.95 \\\hline
  0.5 & 17.95 & 13.93 \\\hline
  0.6 & 24.58 & 15.99 \\\hline
  0.7 & 34.34 & 19.9 \\\hline
  0.8 & 40.2 & 25.63 \\\hline
  0.9 & 54.17 & 28.86 \\\hline
\end{array}

We observe that, whatever the temperature, our DQN_CNN significantly outperforms the DQN_FC. However, both agents have the same issue for every temperature: they do not explore enough the map and turn on themselves passed a certain point. This is even more likely to occur when the agent doesn't have any cheese in its sight or when it is trapped in a corner.

One possible way to improve our models would be to encourage the agent to explore more the map, by maybe rewarding exploration, penalizing the agent when it comes back on a cell it has already explored, or decreasing the cost of exploration over time.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [45]:
def train_explore(agent,env,epoch,decay_parameter_epsilon=0.3,prefix=''):
    #New training procedure that tries to improve the exploration of the algorithm
    #decay_parameter_epsilon in order to use the decreasing $\epsilon$-greedy exploration
    
    
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        agent.set_epsilon(agent.epsilon*(1-decay_parameter_epsilon))
        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size)) #define maluses when going to a previously visited position
        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action,train=False):
        """This function returns the new state, reward and decides if the
        game ends."""
        #During the training phase going back to a position where the rat have already been before tends to decrease the 
        #total reward hence there is this train parameter that we have added (it tries to enforce the exploration)

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1     
        
        ## In Environment exploring:
        # You will have to change n_state to 3 because you will use one more layer!
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1

        reward = reward + self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        #At the begining the malus_position array must be setted to zero
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [46]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore50.mp4'))

Epoch 000/051 | Loss 0.0072 | Win/lose count 3.0/22.200000000000053 (-19.200000000000053)
Epoch 001/051 | Loss 0.0031 | Win/lose count 7.5/18.599999999999994 (-11.099999999999994)
Epoch 002/051 | Loss 0.0227 | Win/lose count 8.0/17.99999999999999 (-9.99999999999999)
Epoch 003/051 | Loss 0.0052 | Win/lose count 5.0/20.200000000000017 (-15.200000000000017)
Epoch 004/051 | Loss 0.0072 | Win/lose count 9.0/16.299999999999965 (-7.299999999999965)
Epoch 005/051 | Loss 0.0023 | Win/lose count 12.0/17.099999999999977 (-5.0999999999999766)
Epoch 006/051 | Loss 0.0051 | Win/lose count 6.5/18.399999999999995 (-11.899999999999995)
Epoch 007/051 | Loss 0.0100 | Win/lose count 8.5/17.099999999999977 (-8.599999999999977)
Epoch 008/051 | Loss 0.0046 | Win/lose count 5.0/17.49999999999998 (-12.499999999999979)
Epoch 009/051 | Loss 0.0233 | Win/lose count 6.5/17.299999999999976 (-10.799999999999976)
Epoch 010/051 | Loss 0.0081 | Win/lose count 3.0/18.699999999999996 (-15.699999999999996)
Epoch 011/051 |

In [47]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore49.mp4'))

Win/lose count 21.5/4.0. Average score (17.5)
Win/lose count 25.0/3.0. Average score (19.75)
Win/lose count 27.0/6.0. Average score (20.166666666666668)
Win/lose count 23.5/2.0. Average score (20.5)
Win/lose count 20.5/1.0. Average score (20.3)
Win/lose count 7.5/0. Average score (18.166666666666668)
Win/lose count 23.0/0. Average score (18.857142857142858)
Win/lose count 24.0/2.0. Average score (19.25)
Win/lose count 19.0/1.0. Average score (19.11111111111111)
Win/lose count 15.0/4.0. Average score (18.3)
Win/lose count 11.0/0. Average score (17.636363636363637)
Win/lose count 24.0/3.0. Average score (17.916666666666668)
Win/lose count 19.5/0. Average score (18.03846153846154)
Win/lose count 21.0/2.0. Average score (18.107142857142858)
Win/lose count 24.0/5.0. Average score (18.166666666666668)
Win/lose count 15.5/4.0. Average score (17.75)
Win/lose count 22.0/3.0. Average score (17.823529411764707)
Win/lose count 28.0/3.0. Average score (18.22222222222222)
Win/lose count 21.5/0. Aver

Encouraging exploration resulted in a great increase in performance, even for a temperature of 0.3, our DQN_CNN has the best score  so far. In fact, at the end, our DQN_CNN has a mean test score of 19, which is close to the perfect game. 

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***